## qflib Library Examples and HW 4 Solution

In [1]:
import qflib as qf
import numpy as np
import os

print("qflib version: {0}".format(qf.version()))
print("pid: {0}".format(os.getpid()))

qflib version: 0.4.0-debug
pid: 5104


In [2]:
name = "World"
print(qf.sayHello(name))

Hello World!


In [3]:
x = [1, 2, 3]
y = [4, 5]
op = qf.outerProd(x, y)
print(f'x: {x}\ny: {y}')
print(f'outerProd:\n{op}')

x: [1, 2, 3]
y: [4, 5]
outerProd:
[[ 4.  5.]
 [ 8. 10.]
 [12. 15.]]


In [4]:
m = np.array([[1, 2, 3], [4, 5, 6]])
em = qf.echoMatrix(m)
print(f'orig matrix:\n{m}')
print(f'echo matrix:\n{em}')

orig matrix:
[[1 2 3]
 [4 5 6]]
echo matrix:
[[1. 2. 3.]
 [4. 5. 6.]]


### Analytic Prices

In [5]:
#fwdprice
fwdpx = qf.fwdPrice(spot = 100, timetoexp = 1.0, intrate = 0.02, divyield = 0.04)
print('Forward price analytic solution')
print(f'Price={fwdpx:.4f}')

Forward price analytic solution
Price=98.0199


In [6]:
#digibs
digi = qf.digiBS(payofftype = 1, spot = 100, timetoexp = 1.0, strike = 100,
                  intrate = 0.02, divyield = 0.04, volatility = 0.2)
print('Digital option using Black-Scholes analytic solution')
print(f'Price={digi:}')

Digital option using Black-Scholes analytic solution
Price=[4.12409075e-01 1.87854845e+00 3.75709691e-03 3.01392571e+00
 4.25547963e-16]


In [7]:
euro = qf.euroBS(payofftype = 1, spot = 100, timetoexp = 1.0, strike = 100,
                  intrate = 0.02, divyield = 0.04, volatility = 0.4)
print('European option using Black-Scholes analytic solution')
print(f'Price={euro:}')

European option using Black-Scholes analytic solution
Price=[ 1.44327242e+01  5.37674769e-01  9.47528938e-03 -6.21622748e+00
  3.79011575e+01]


In [8]:
import random
import pandas as pd
#1) CREATE RANDOM PARAMETER DATAFRAME 

data = []
for i in range(10):
    payoff    = random.choice([1, -1])  # Must be int
    spot      = random.uniform(80, 120)
    strike    = 100.0
    timetoexp = random.uniform(0.3, 2.0)
    intrate   = random.uniform(0.0, 0.05)
    divyield  = random.uniform(0.0, 0.03)
    vol       = random.uniform(0.2, 0.5)

    data.append((payoff, spot, strike, timetoexp, intrate, divyield, vol))

df = pd.DataFrame(data, columns=[
    'Payoff','Spot','Strike','TimeToExp','IntRate','DivYield','Volatility'
])

# 'Payoff' is actually int
df['Payoff'] = df['Payoff'].astype(int)

# 2) ADD COLUMNS FOR PRICE, DELTA, GAMMA, THETA, VEGA 

df['Price'] = np.nan
df['Delta'] = np.nan
df['Gamma'] = np.nan
df['Theta'] = np.nan
df['Vega']  = np.nan


In [9]:
# 3) CALL qf.euroBS FOR EACH ROW, FILL GREEKS  

for i, row in df.iterrows():
    payoff   = int(row['Payoff'])   # ensure integer
    S        = row['Spot']
    K        = row['Strike']
    T        = row['TimeToExp']
    r        = row['IntRate']
    q_       = row['DivYield']
    vol      = row['Volatility']

    # qf.euroBS => np.array([Price, Delta, Gamma, Theta, Vega])
    greeks = qf.euroBS(payoff, S, K, T, r, q_, vol)

    df.at[i, 'Price'] = greeks[0]
    df.at[i, 'Delta'] = greeks[1]
    df.at[i, 'Gamma'] = greeks[2]
    df.at[i, 'Theta'] = greeks[3]
    df.at[i, 'Vega']  = greeks[4]

# 4) COMPUTE THE "Diff" COLUMN 
df['Diff'] = (
    df['Theta']
    + (df['IntRate'] - df['DivYield']) * df['Delta'] * df['Spot']
    + 0.5 * (df['Volatility']**2) * (df['Spot']**2) * df['Gamma']
    - df['IntRate'] * df['Price']
)

df.head(10)


,Payoff,Spot,Strike,TimeToExp,IntRate,DivYield,Volatility,Price,Delta,Gamma,Theta,Vega,Diff
0,-1,98.560578,100.0,0.344214,0.031308,0.001936,0.472882,11.108617,-0.450792,0.014470,-14.063597,22.880123,-4.440892e-16
1,1,110.757320,100.0,1.904722,0.001676,0.008601,0.409092,28.106368,0.659108,0.005697,-5.295570,54.458212,-2.498002e-16
2,-1,94.365796,100.0,1.752157,0.041013,0.029639,0.204404,11.662417,-0.476739,0.014834,-1.769544,47.309822,1.665335e-16
3,-1,90.166746,100.0,0.874702,0.000570,0.004062,0.262401,15.174336,-0.620090,0.017114,-4.976708,31.935257,-6.505213e-16
4,1,87.663393,100.0,1.314785,0.013978,0.029663,0.470184,13.203899,0.475925,0.008118,-6.056529,38.563969,-4.996004e-16
5,-1,118.856919,100.0,0.909098,0.044274,0.029109,0.421970,8.788195,-0.246479,0.006514,-7.359155,35.300102,1.665335e-16
6,-1,110.499896,100.0,1.793061,0.001040,0.028218,0.297275,13.673937,-0.353333,0.008172,-5.455999,53.188131,-7.112366e-17
7,1,115.693857,100.0,0.936571,0.031379,0.006795,0.372319,25.699027,0.737009,0.007705,-8.438205,35.963846,7.771561e-16
8,-1,100.228583,100.0,0.960692,0.046425,0.009679,0.219319,6.637969,-0.385248,0.017628,-2.531947,37.311880,-2.220446e-16
9,1,104.326973,100.0,1.426375,0.045091,0.008470,0.357184,21.675968,0.660209,0.008057,-7.138760,44.676487,0.000000e+00


In [10]:
data2 = []
for i in range(10):
    payoff    = random.choice([1, -1])  # Must be int: +1 = digital call, -1 = digital put
    spot      = random.uniform(80, 120)
    strike    = 100.0
    timetoexp = random.uniform(0.3, 2.0)
    intrate   = random.uniform(0.0, 0.05)
    divyield  = random.uniform(0.0, 0.03)
    vol       = random.uniform(0.2, 0.5)

    data2.append((payoff, spot, strike, timetoexp, intrate, divyield, vol))

df2 = pd.DataFrame(data2, columns=[
    'Payoff','Spot','Strike','TimeToExp','IntRate','DivYield','Volatility'
])

# Ensure 'Payoff' is actually int 
df2['Payoff'] = df2['Payoff'].astype(int)

# 2) ADD COLUMNS FOR Price, Delta, Gamma, Theta, Vega
df2['Price'] = np.nan
df2['Delta'] = np.nan
df2['Gamma'] = np.nan
df2['Theta'] = np.nan
df2['Vega']  = np.nan

# 3) CALL qf.digiBS FOR EACH ROW, FILL GREEKS  
for i, row in df2.iterrows():
    payoff   = int(row['Payoff'])
    S        = row['Spot']
    K        = row['Strike']
    T        = row['TimeToExp']
    r        = row['IntRate']
    q_       = row['DivYield']
    vol      = row['Volatility']

    # digiBS => np.array([Price, Delta, Gamma, Theta, Vega])
    greeks = qf.digiBS(payoff, S, K, T, r, q_, vol)

    df2.at[i, 'Price'] = greeks[0]
    df2.at[i, 'Delta'] = greeks[1]
    df2.at[i, 'Gamma'] = greeks[2]
    df2.at[i, 'Theta'] = greeks[3]
    df2.at[i, 'Vega']  = greeks[4]

# 4) COMPUTE THE "Diff" COLUMN       
df2['Diff'] = (
    df2['Theta']
    + (df2['IntRate'] - df2['DivYield']) * df2['Delta'] * df2['Spot']
    + 0.5 * (df2['Volatility']**2) * (df2['Spot']**2) * df2['Gamma']
    - df2['IntRate'] * df2['Price']
)

df.head(10)

,Payoff,Spot,Strike,TimeToExp,IntRate,DivYield,Volatility,Price,Delta,Gamma,Theta,Vega,Diff
0,-1,98.560578,100.0,0.344214,0.031308,0.001936,0.472882,11.108617,-0.450792,0.014470,-14.063597,22.880123,-4.440892e-16
1,1,110.757320,100.0,1.904722,0.001676,0.008601,0.409092,28.106368,0.659108,0.005697,-5.295570,54.458212,-2.498002e-16
2,-1,94.365796,100.0,1.752157,0.041013,0.029639,0.204404,11.662417,-0.476739,0.014834,-1.769544,47.309822,1.665335e-16
3,-1,90.166746,100.0,0.874702,0.000570,0.004062,0.262401,15.174336,-0.620090,0.017114,-4.976708,31.935257,-6.505213e-16
4,1,87.663393,100.0,1.314785,0.013978,0.029663,0.470184,13.203899,0.475925,0.008118,-6.056529,38.563969,-4.996004e-16
5,-1,118.856919,100.0,0.909098,0.044274,0.029109,0.421970,8.788195,-0.246479,0.006514,-7.359155,35.300102,1.665335e-16
6,-1,110.499896,100.0,1.793061,0.001040,0.028218,0.297275,13.673937,-0.353333,0.008172,-5.455999,53.188131,-7.112366e-17
7,1,115.693857,100.0,0.936571,0.031379,0.006795,0.372319,25.699027,0.737009,0.007705,-8.438205,35.963846,7.771561e-16
8,-1,100.228583,100.0,0.960692,0.046425,0.009679,0.219319,6.637969,-0.385248,0.017628,-2.531947,37.311880,-2.220446e-16
9,1,104.326973,100.0,1.426375,0.045091,0.008470,0.357184,21.675968,0.660209,0.008057,-7.138760,44.676487,0.000000e+00
